In [ ]:
!pip install -q transformers==4.6.1 tensorflow==2.4.1

     |████████████████████████████████| 394.3MB 39kB/s 
     |████████████████████████████████| 3.8MB 37.5MB/s 
     |████████████████████████████████| 471kB 33.2MB/s 


In [ ]:
from transformers import TFBertModel, BertTokenizer 

MODEL_NAME = 'bert-base-cased'

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME).save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))
model = TFBertModel.from_pretrained(MODEL_NAME).save_pretrained("./{}".format(MODEL_NAME), saved_model=True)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


INFO:tensorflow:Assets written to: ./bert-base-cased/saved_model/1/assets


INFO:tensorflow:Assets written to: ./bert-base-cased/saved_model/1/assets


In [ ]:
!ls -l {MODEL_NAME}

total 423352
-rw-r--r-- 1 root root       597 May 30 12:45 config.json
drwxr-xr-x 3 root root      4096 May 30 12:45 saved_model
-rw-r--r-- 1 root root 433503400 May 30 12:45 tf_model.h5


In [ ]:
!ls -l {MODEL_NAME}/saved_model/1

total 7768
drwxr-xr-x 2 root root    4096 May 30 12:45 assets
-rw-r--r-- 1 root root 7944426 May 30 12:45 saved_model.pb
drwxr-xr-x 2 root root    4096 May 30 12:45 variables


In [ ]:
!ls -l {MODEL_NAME}_tokenizer

total 220
-rw-r--r-- 1 root root    112 May 30 12:44 special_tokens_map.json
-rw-r--r-- 1 root root    521 May 30 12:44 tokenizer_config.json
-rw-r--r-- 1 root root 213450 May 30 12:44 vocab.txt


In [ ]:
!cp {MODEL_NAME}_tokenizer/vocab.txt {MODEL_NAME}/saved_model/1/assets

In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

In [ ]:
from sparknlp.annotator import *

bert = BertEmbeddings.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["sentence",'token'])\
 .setOutputCol("bert")\
 .setCaseSensitive(True)\
 .setDimension(768)\
 .setStorageRef('bert_base_cased') 

In [ ]:
bert.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

In [ ]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 393292
-rw-r--r-- 1 root root 402718696 May 30 13:04 bert_tensorflow
drwxr-xr-x 4 root root      4096 May 30 13:02 fields
drwxr-xr-x 2 root root      4096 May 30 13:02 metadata


In [ ]:
bert_loaded = BertEmbeddings.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["sentence",'token'])\
  .setOutputCol("bert")\
  .setCaseSensitive(True)

In [ ]:
bert_loaded.getStorageRef()

'bert_base_cased'